In [52]:
%matplotlib inline

import os, subprocess, tarfile
from glob import glob
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cftime
import cartopy
import cartopy.crs as ccrs
import scipy.stats as stats
from scipy.io import loadmat

import string
alphabet=list(string.ascii_lowercase)

import warnings
warnings.filterwarnings('ignore')

In [53]:
import matplotlib.colors as colors
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Read in Data

In [54]:
case={}

#control
case['path_root']='/archive/Jessica.Luo/gz_test/MOM6_SIS2_COBALT'
case['machine_target']='gfdl.ncrc4-intel19-prod'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-013022_cy5'

#cases
case['path_root']='/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2'
case['machine_target']='gfdl.ncrc4-intel19-prod'

case['name']='OM4p5_CORE2_IAF_gzCOBALT-tun_cy5'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-tun_hp_cy5'
case['name']='OM4p5_CORE2_IAF_gzCOBALT-hp_cy5'


In [55]:
# define years for cycle
cycle=1
if 'cy1' in case['name']:
    cycle=1
if 'cy2' in case['name']:
    cycle=2
if 'cy3' in case['name']:
    cycle=3
if 'cy4' in case['name']:
    cycle=4
if 'cy5' in case['name']:
    cycle=5
    
print(cycle)

5


In [56]:
start_date=60*(cycle-1)+1
end_date=60*cycle+1
print(start_date,end_date)

241 301


In [57]:
short_casename=case['name'].replace('OM4p5_CORE2_IAF_','')
if short_casename=="gzCOBALT-060822":
    fig_casename='gzCOBALT-tun_cy1'
else:
    fig_casename=short_casename
print(short_casename, fig_casename)

gzCOBALT-hp_cy5 gzCOBALT-hp_cy5


In [58]:
# read in files
diagType = 'ocean_cobalt_omip_tracers_year_z'
varlist = ['o2','volcello']

prefix = os.path.join(case['path_root'], case['name'], case['machine_target'], 'pp', diagType, 'ts', 'annual', '5yr', diagType)
files_a = glob('.'.join([prefix, '*', 'o2', 'nc']))
files_b = glob('.'.join([prefix, '*', 'volcello', 'nc']))
files_a.extend(files_b)
print(files_a)

['/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2/OM4p5_CORE2_IAF_gzCOBALT-hp_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/ocean_cobalt_omip_tracers_year_z.1948-1952.o2.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2/OM4p5_CORE2_IAF_gzCOBALT-hp_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/ocean_cobalt_omip_tracers_year_z.1953-1957.o2.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2/OM4p5_CORE2_IAF_gzCOBALT-hp_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/ocean_cobalt_omip_tracers_year_z.1958-1962.o2.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2/OM4p5_CORE2_IAF_gzCOBALT-hp_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/ocean_cobalt_omip_tracers_year_z.1963-1967.o2.nc', '/archive/Jessica.Luo/gz_test/MOM6_SIS2_gzCOBALTv2/OM4p5_CORE2_IAF_gzCOBALT-hp_cy5/gfdl.ncrc4-intel19-prod/pp/ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/ocean_coba

In [59]:
grid_file=os.path.join(case['path_root'], case['name'], case['machine_target'], 'pp')+'/ocean_annual/ocean_annual.static.nc'
ds=xr.open_mfdataset(files_a, combine='by_coords', decode_times=False)
grid=xr.open_dataset(grid_file)

In [60]:
ds['depth_levels']=xr.DataArray(ds.z_i.diff(dim='z_i').values, coords={'z_l':ds.z_l.values}, dims=['z_l'], attrs={'long_name':'Thickness of depth bin', 'units':'meters'})

# Compute Hypoxic Volume - depth integral

In [61]:
sub_60mmol = ds.volcello.where(ds.o2 <= 0.06).sum(dim=['z_l'])
sub_5mmol = ds.volcello.where(ds.o2 <= 0.005).sum(dim=['z_l'])

In [62]:
da_sub_60mmol = xr.DataArray(sub_60mmol.values, coords={'time':np.arange(start_date,end_date),
                                                        'yh':ds.yh.values,
                                                        'xh':ds.xh.values}, 
                             dims=['time','yh','xh'], name='sub_60mmol', 
                             attrs={'name':'total hypoxic volume (where O2 < 60mmol)','units':'cubic meters'})
da_sub_5mmol = xr.DataArray(sub_5mmol.values, coords={'time':np.arange(start_date,end_date),
                                                        'yh':ds.yh.values,
                                                        'xh':ds.xh.values}, 
                             dims=['time','yh','xh'], name='sub_5mmol', 
                            attrs={'name':'total suboxic volume (where O2 < 5mmol)','units':'cubic meters'})

In [63]:
hypoxic_vol_spatial_ts = da_sub_60mmol.to_dataset()
hypoxic_vol_spatial_ts['sub_5mmol'] = da_sub_5mmol

hypoxic_vol_spatial_ts

<xarray.Dataset>
Dimensions:     (time: 60, xh: 720, yh: 576)
Coordinates:
  * time        (time) int64 241 242 243 244 245 246 ... 295 296 297 298 299 300
  * yh          (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * xh          (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
Data variables:
    sub_60mmol  (time, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    sub_5mmol   (time, yh, xh) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [64]:
file_out='data/hypoxicVolume_Spatial.'+fig_casename+'.nc'

if not os.path.exists(file_out):
    hypoxic_vol_spatial_ts.to_netcdf(path=file_out, mode='w')
    print('writing to: '+file_out)
else:
    print(file_out + ' exists')

writing to: data/hypoxicVolume_Spatial.gzCOBALT-hp_cy5.nc


# Compute Hypoxic Volume - global sum

In [65]:
sub_60mmol = ds.volcello.where(ds.o2 <= 0.06).sum(dim=['xh','yh','z_l'])
sub_5mmol = ds.volcello.where(ds.o2 <= 0.005).sum(dim=['xh','yh','z_l'])

In [66]:
da_sub_60mmol = xr.DataArray(sub_60mmol.values, coords={'time':np.arange(start_date,end_date)}, dims=['time'], name='sub_60mmol', 
                             attrs={'name':'total hypoxic volume (where O2 < 60mmol)','units':'cubic meters'})
da_sub_5mmol = xr.DataArray(sub_5mmol.values, coords={'time':np.arange(start_date,end_date)}, dims=['time'], name='sub_60mmol', 
                            attrs={'name':'total suboxic volume (where O2 < 5mmol)','units':'cubic meters'})

In [67]:
hypoxic_vol_ts = da_sub_60mmol.to_dataset()
hypoxic_vol_ts['sub_5mmol'] = da_sub_5mmol

hypoxic_vol_ts

<xarray.Dataset>
Dimensions:     (time: 60)
Coordinates:
  * time        (time) int64 241 242 243 244 245 246 ... 295 296 297 298 299 300
Data variables:
    sub_60mmol  (time) float32 9.212e+16 9.215e+16 ... 9.225e+16 9.227e+16
    sub_5mmol   (time) float32 2.019e+16 2.012e+16 ... 2.071e+16 2.072e+16

In [68]:
file_out='data/hypoxicVolume.'+fig_casename+'.nc'

if not os.path.exists(file_out):
    hypoxic_vol_ts.to_netcdf(path=file_out, mode='w')
    print('writing to: '+file_out)
else:
    print(file_out + ' exists')

writing to: data/hypoxicVolume.gzCOBALT-hp_cy5.nc
